In [1]:
from pathlib import Path
import torch
import torch.nn as nn
from config import get_config, get_weights_file_path
from train import get_model, get_ds_csv, run_validation
from train import translate_input_string, translate_input_string_alt, eos_add, translate_input_string_test

C:\Users\oscar\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\oscar\AppData\Roaming\Python\Python311\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [83]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
config = get_config()
# config['model_basename'] = "tmodel_eight_four_id_"
config['model_basename'] = "tmodel_new_ma_"
# Training bahasa target menjadi bahasa Maanyan
# config['lang_src'] = "Indonesia"
# config['lang_tgt'] = "Maanyan"
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds_csv(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = get_weights_file_path(config, f"29")
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Using device: cuda
Max length of source sentence: 22
Max length of target sentence: 24


<All keys matched successfully>

In [9]:
input_string = "hanyu manre"

# translation = translate_input_string_test(model, input_string, tokenizer_src, tokenizer_tgt, config['seq_len'], device)
translation = translate_input_string_test(model, input_string, tokenizer_src, tokenizer_tgt, config['seq_len'], device)


AttributeError: 'str' object has no attribute 'len'

In [52]:
input_string = "hanyu tulak ma palangka"
expect = "kamu"

print(len(input_string.split(" ")))

# input_string = eos_add(input_string)

# print(input_string)
# translation = translate_input_string(model, input_string, tokenizer_src, tokenizer_tgt, config['seq_len'], device)
translation = translate_input_string(model, input_string, tokenizer_src, tokenizer_tgt, len(input_string.split(" "))+1, device)
print(f"Input String    : '{input_string}'")
# print(f"Output String   : '{expect}'")
print(f"Translation     : '{translation}'")

4
Input String    : 'hanyu tulak ma palangka'
Translation     : 'kamu pergi ke palangka'


In [30]:
def add_eos(string):
    return ""+ string + ""

# Menggunakan fungsi translate_input_string

# input_string = add_eos("aku") 
input_string = ["dia di mana", "kamu makan durian", "aku pergi ke palangka"]
output_string = ["hanye hang awe", "hanyu nguta ruyan", 'aku tulak ma palangka']
translation = translate_input_string(model, input_string[0], tokenizer_src, tokenizer_tgt, len(input_string[0].split(" "))+1, device)
print(f"Input String    : '{input_string[0]}'")
print(f"Output String   : '{output_string[0]}'")
print(f"Translation     : '{translation}'")
print('===============================================')
translation = translate_input_string(model, input_string[1], tokenizer_src, tokenizer_tgt, len(input_string[1].split(" "))+1, device)
print(f"Input String    : '{input_string[1]}'")
print(f"Output String   : '{output_string[1]}'")
print(f"Translation     : '{translation}'")
print('===============================================')
translation = translate_input_string(model, input_string[2], tokenizer_src, tokenizer_tgt, len(input_string[2].split(" "))+1, device)
print(f"Input String    : '{input_string[2]}'")
print(f"Output String   : '{output_string[2]}'")
print(f"Translation     : '{translation}'")

Input String    : 'dia di mana'
Output String   : 'hanye hang awe'
Translation     : 'hanye na hang'
Input String    : 'kamu makan durian'
Output String   : 'hanyu nguta ruyan'
Translation     : 'hanyu nguta ruyan'
Input String    : 'aku pergi ke palangka'
Output String   : 'aku tulak ma palangka'
Translation     : 'aku tulak ma palangka'


In [4]:
jika = "kamu makan."

print(eos_add(jika))

kamu makan [EOS]


In [4]:
from nltk.translate.bleu_score import sentence_bleu

In [82]:
reference = [
    "hanyu nguta ruyan hang ina".split(),
    "hanyu kuman ruyan hang ina".split(),
]

input_string = 'kamu makan durian di sini'

test = translate_input_string(model, input_string, tokenizer_src, tokenizer_tgt, config['seq_len'], device)
# test = translate_input_string(model, input_string, tokenizer_src, tokenizer_tgt,  len(input_string.split(" "))+1, device)
score = sentence_bleu(reference, test.split(), weights=(1, 0, 0, 0))

print(f'input text : {input_string}')
print(f'translated text : {test}')
print(f'BLEU Score : {score}')

input text : kamu makan durian di sini
translated text : hanyu nguta ruyan hang ruyan na
BLEU Score : 0.6666666666666666


In [94]:
reference = [
    "kamu pergi ke palangka".split(),
]

input_string = 'hanyu tulak ma palangka'

test = translate_input_string(model, input_string, tokenizer_src, tokenizer_tgt, config['seq_len'], device)
# test = translate_input_string(model, input_string, tokenizer_src, tokenizer_tgt,  len(input_string.split(" "))+1, device)
score = sentence_bleu(reference, test.split(), weights=(1, 0, 0, 0))

print(f'input text : {input_string}')
print(f'translated text : {test}')
print(f'BLEU Score : {score}')

input text : hanyu tulak ma palangka
translated text : kamu pergi ke palangka palangka hari palangka
BLEU Score : 0.5714285714285714


In [7]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=10)

--------------------------------------------------------------------------------
    SOURCE: dan di sana ia memanggil nama tuhan allah yang kekal
    TARGET: nelang hang yaru hanye nanterau ngaran  tuhan  alatalla sa kalalawah
 PREDICTED: nelang hang yaru hanye ngaran tuhan alatalla sa
--------------------------------------------------------------------------------
    SOURCE: perut dan pinggangnya dari tembaga
    TARGET: wuntung anri pingangni teka tambaga
 PREDICTED: anri teka tambaga
--------------------------------------------------------------------------------
    SOURCE: mereka menetap berhadapan dengan semua saudaranya
    TARGET: here muneng sawung hadap anri kawan pulaksanaini
 PREDICTED: here muneng anri kawan
--------------------------------------------------------------------------------
    SOURCE: memberkati dia serta berpesan kepadanya
    TARGET: mamberkat nelang ipesen ma hanye
 PREDICTED: mamberkat nelang ma hanye
----------------------------------------------------